<a href="https://colab.research.google.com/github/vajja1405/Facial-Recognition/blob/main/Facial_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python matplotlib numpy

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torchvision import datasets, models
from torch.utils.data import DataLoader


In [4]:
!unzip archive.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/data/train/6/28451.png  
  inflating: dataset/data/train/6/28452.png  
  inflating: dataset/data/train/6/28467.png  
  inflating: dataset/data/train/6/28471.png  
  inflating: dataset/data/train/6/28472.png  
  inflating: dataset/data/train/6/28478.png  
  inflating: dataset/data/train/6/2848.png  
  inflating: dataset/data/train/6/28480.png  
  inflating: dataset/data/train/6/28484.png  
  inflating: dataset/data/train/6/28485.png  
  inflating: dataset/data/train/6/28493.png  
  inflating: dataset/data/train/6/28499.png  
  inflating: dataset/data/train/6/28507.png  
  inflating: dataset/data/train/6/28517.png  
  inflating: dataset/data/train/6/28519.png  
  inflating: dataset/data/train/6/28520.png  
  inflating: dataset/data/train/6/28524.png  
  inflating: dataset/data/train/6/28530.png  
  inflating: dataset/data/train/6/28531.png  
  inflating: dataset/data/train/6/28535.png  
  inflating: dataset/data/trai

In [14]:
import os
print(os.listdir("dataset/data"))


['train', 'val', 'test']


In [15]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.Resize((48, 48)),  # Resize images to 48x48
    transforms.RandomHorizontalFlip(),  # Random flipping for data augmentation
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values
])

In [17]:
train_data = datasets.ImageFolder(root="dataset/data/train", transform=transform)
test_data = datasets.ImageFolder(root="dataset/data/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [18]:
model = models.resnet18(pretrained=True)  # Load pretrained ResNet18
model.fc = nn.Linear(512, 7)  # Modify the last layer for 7 emotion classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU if available


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.1MB/s]


In [19]:
criterion = nn.CrossEntropyLoss()  # Classification loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [20]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)


Epoch 1/5, Loss: 1.3900048990292113
Epoch 2/5, Loss: 1.2007536654151096
Epoch 3/5, Loss: 1.138634542513531
Epoch 4/5, Loss: 1.1307202008094448
Epoch 5/5, Loss: 1.043888247690116


In [21]:
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model, test_loader)


Accuracy: 56.90%


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import os
import numpy as np
from PIL import Image

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data Augmentation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small translations
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Simulate real-world conditions
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

# Load dataset
data_dir = "dataset/data"  # Adjust if necessary
train_data = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
test_data = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Load Pretrained Model (ResNet50)
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 7)  # Adjust final layer for 7 emotion classes
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Reduce LR every 5 epochs

# Training Function
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f} - Accuracy: {correct/total:.2%}")

# Train the model
train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20)

# Test Function with Test-Time Augmentation (TTA)
def predict_with_tta(model, image_path, transform, n_augmentations=5):
    model.eval()
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    augmented_images = [transform(image) for _ in range(n_augmentations)]
    augmented_images = torch.stack(augmented_images).to(device)

    with torch.no_grad():
        outputs = model(augmented_images)
        final_prediction = torch.mean(outputs, dim=0)  # Average predictions

    return torch.argmax(final_prediction).item()

# Save Model
torch.save(model.state_dict(), "facial_recognition_model.pth")
print("Model saved successfully!")


Using device: cuda
Epoch 1/20 - Loss: 1.4871 - Accuracy: 42.97%
Epoch 2/20 - Loss: 1.2468 - Accuracy: 52.51%
Epoch 3/20 - Loss: 1.1950 - Accuracy: 54.71%
Epoch 4/20 - Loss: 1.1493 - Accuracy: 56.10%
Epoch 5/20 - Loss: 1.1162 - Accuracy: 57.96%
Epoch 6/20 - Loss: 1.0336 - Accuracy: 61.00%
Epoch 7/20 - Loss: 1.0056 - Accuracy: 61.81%
Epoch 8/20 - Loss: 0.9894 - Accuracy: 62.50%
Epoch 9/20 - Loss: 0.9864 - Accuracy: 62.45%
Epoch 10/20 - Loss: 0.9640 - Accuracy: 63.64%
Epoch 11/20 - Loss: 0.9155 - Accuracy: 65.45%
Epoch 12/20 - Loss: 0.8959 - Accuracy: 66.24%
Epoch 13/20 - Loss: 0.8761 - Accuracy: 66.59%
Epoch 14/20 - Loss: 0.8719 - Accuracy: 67.37%
Epoch 15/20 - Loss: 0.8541 - Accuracy: 67.74%
Epoch 16/20 - Loss: 0.8180 - Accuracy: 69.26%
Epoch 17/20 - Loss: 0.8004 - Accuracy: 70.06%
Epoch 18/20 - Loss: 0.7839 - Accuracy: 70.63%
Epoch 19/20 - Loss: 0.7747 - Accuracy: 71.05%
Epoch 20/20 - Loss: 0.7680 - Accuracy: 71.06%
Model saved successfully!
